In [3]:
import math
import copy

In [15]:
fread = open("Q2Q2.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split('\t') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split('\t') #change to \t if tab spaced
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = int(t_h[3].strip())
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

cost = [[math.inf for i in range(V)] for j in range(V)]
ratio = [[0 for i in range(V)] for j in range(V)]
time = [[math.inf for i in range(V)] for j in range(V)]
profit = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][3]/G_mat[i][2]
    ratio[tail][head] = G_mat[i][2]/G_mat[i][3]
    time[tail][head] = G_mat[i][3]
    profit[tail][head] = G_mat[i][2]

print(G_mat, "\n")
print(A_dict)   
print(cost)
fread.close()

[[0, 1, 200, 30], [1, 0, 150, 20], [0, 2, 200, 35], [1, 2, 100, 20], [2, 1, 250, 40], [1, 4, 300, 40], [2, 3, 225, 50], [3, 2, 350, 40], [3, 4, 275, 25], [4, 0, 200, 40], [4, 2, 250, 35]] 

{0: [1, 2], 1: [0, 2, 4], 2: [1, 3], 3: [2, 4], 4: [0, 2]}
[[inf, 0.15, 0.175, inf, inf], [0.13333333333333333, inf, 0.2, inf, 0.13333333333333333], [inf, 0.16, inf, 0.2222222222222222, inf], [inf, inf, 0.11428571428571428, inf, 0.09090909090909091], [0.2, inf, 0.14, inf, inf]]


In [16]:
#Algo 2 Floyd Warshall O(n^3)
def FloydWarshall(cost):
    D = copy.deepcopy(cost)
    path = [[]]*(V*V)
    for i in range(V):
        for j in range(V):
            if ((i != j) and cost[i][j] != math.inf):
                index = V*i + j
                path[index] = [[i,j]]
    for k in range(V):
        for i in range(V):
            if (i == k):
                continue
            for j in range(V):
                if (j == k):
                    continue
                upd = False
                if (D[i][j] > D[i][k] + D[k][j]):
                    upd = True
                D[i][j] = min(D[i][j], D[i][k] + D[k][j])
                if (upd):
                    index = V*i + j
                    path[index] = path[V*i + k] + path[V*k + j]
    return D, path

min_cost_all, path = FloydWarshall(cost)
#print(min_cost_all)
s = 0
t = 4
print(min_cost_all[s][t])
print(path[s*V + t])                           
                

0.2833333333333333
[[0, 1], [1, 4]]


In [18]:
def upd_cost(alp, time, profit, V):
    c1 = [[math.inf for i in range(V)] for j in range(V)]
    for i in range(V):
        for j in range(V):
            if(time[i][j] == math.inf):
                continue
            c1[i][j] = alp*time[i][j] - profit[i][j]
    return c1

max_ratio = 0
min_ratio = math.inf
#print(ratio)
for i in range(V):
    for j in range(V):
        if (ratio[i][j] > max_ratio):
            max_ratio = ratio[i][j]
        if (ratio[i][j] < min_ratio and ratio[i][j] != 0):
            min_ratio = ratio[i][j]

alp = (max_ratio + min_ratio)/2
#print(alp)

while (max_ratio - min_ratio > .01): #precision value
    #Find negative cost cycle
    #print('start')
    #print(alp)
    new_cost = upd_cost(alp, time, profit, V)
    #print(new_cost)
    min_cost_all, path = FloydWarshall(new_cost)
    #print(min_cost_all)
    # check for negative cycle
    neg_flag = 0
    for i in range(V):
        if (min_cost_all[i][i] < 0):
            neg_flag = 1
            break
    if (neg_flag):
        min_ratio = alp
        alp = (alp + max_ratio)/2
    else:
        max_ratio = alp
        alp = (alp + min_ratio)/2
    #print(max_ratio)
    #print(min_ratio)
print('attractiveness: ', alp)
print('path: ', path[i*V + i])
#print(min_cost_all)


    

attractiveness:  6.997802734375
path:  [[0, 1], [1, 0]]


In [19]:
#Algo 2 modified Floyd Warshall O(n^3) - (Alternate approach) not working
def m_FloydWarshall(cost, time, profit):
    D = copy.deepcopy(cost)
    path = [[]]*(V*V)
    for i in range(V):
        for j in range(V):
            if ((i != j) and cost[i][j] != math.inf):
                index = V*i + j
                path[index] = [[i,j]]
    for k in range(V):
        for i in range(V):
            if (i == k):
                continue
            for j in range(V):
                if (j == k):
                    continue
                if (time[i][k] == math.inf or time[k][j] == math.inf):
                    continue
                upd = False
                if (D[i][j] > (time[i][k] + time[k][j])/(profit[i][k] + profit[k][j])):
                    upd = True
                    D[i][j] = (time[i][k] + time[k][j])/(profit[i][k] + profit[k][j])
                    profit[i][j] = profit[i][k] + profit[k][j]
                    time[i][j] = time[i][k] + time[k][j]
                if (upd):
                    index = V*i + j
                    path[index] = path[V*i + k] + path[V*k + j]
    return D, path

#print('D matrix')
#for i in range(V):
#    print(cost[i])
min_cost_all, path = m_FloydWarshall(cost, time, profit)
#print('Min cost matrix')
#for i in range(V):
#    print(min_cost_all[i])

snode = -1
min_val = math.inf
for i in range(V):
    if (min_cost_all[i][i] < min_val):
        min_val = min_cost_all[i][i]
        snode = i
print('attractiveness: ', 1/min_val)
print('path: ', path[snode*V + snode])
    


attractiveness:  7.0
path:  [[0, 1], [1, 0]]
